In [ ]:
import pandas as pd
import akshare as ak
from datetime import datetime, timedelta
import os

def update_futures_csv(variety_code: str, variety_name: str, folder="F:/国债期货数据/"):
    file_path = os.path.join(folder, f"{variety_name}_国债期货.csv")
    os.makedirs(folder, exist_ok=True)

    # 读取已有数据
    if os.path.exists(file_path):
        old_df = pd.read_csv(file_path, parse_dates=["date"])
        last_date = old_df["date"].max()
        print(f"📁 {variety_name}：已有数据截至 {last_date.date()}")
    else:
        old_df = pd.DataFrame()
        last_date = datetime(2010, 1, 1)
        print(f"📁 {variety_name}：无历史文件，将从头抓取")

    # 日期范围设置
    next_date = last_date + timedelta(days=1)
    today = datetime.today()

    if next_date >= today:
        print(f"✅ {variety_name} 已是最新，无需更新。")
        return

    start_str = next_date.strftime("%Y%m%d")
    end_str = today.strftime("%Y%m%d")
    print(f"📥 正在抓取 {variety_name} 数据：{start_str} 至 {end_str}")

    try:
        new_df = ak.get_futures_daily(start_date=start_str, end_date=end_str, market="CFFEX")
        new_df = new_df[new_df["variety"] == variety_code].copy()
        if new_df.empty:
            print(f"⚠️ {variety_name}：无新数据可更新")
            return
    except Exception as e:
        print(f"❌ 抓取失败：{e}")
        return

    # 合并新旧数据
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
    combined_df.drop_duplicates(subset=["symbol", "date"], inplace=True)
    combined_df.sort_values(by=["date", "symbol"], inplace=True)
    combined_df.to_csv(file_path, index=False, encoding="utf-8-sig")

    print(f"✅ {variety_name} 更新完成，共 {len(combined_df)} 条记录\n")


if __name__ == "__main__":
    print("📈 国债期货数据自动更新任务启动...\n")
    update_futures_csv("TF", "TF_5年期")
    update_futures_csv("T", "T_10年期")
    update_futures_csv("TS", "TS_2年期")
    update_futures_csv("TL", "TL_30年期")
    print("✅ 所有更新任务完成。")